In [2]:
## Parameters specific to where your folders are and your data
parameter_file = 'params/params.yaml'
import yaml
import sys
import glob
with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
annotation= doc['annotation']
#annotation_gtf = doc['annotation_gtf']
tss_file = doc['tss_annotation']

tss_annotation = doc['tss_annotation']


import os

import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm

from os.path import basename
##mpl.use('Agg')
#mpl.style.use('ggplot')
#mpl.style.use('fivethirtyeight')
from Homer import *

print('Number of tissues: ',len(tissues))

('Number of tissues: ', 13)


In [3]:
import Homer

### Plot the CDF of fraction of genes covered

In [4]:
def to_collapse(curr,collapse_mapping):
    collapse = set()
    for i in curr: 
        collapse.add(collapse_mapping[i])
    return list(collapse)

def create_binary(df,tissues,colname="tissues"):
    bin_df = pd.DataFrame(index=df.index,columns=tissues,dtype=bool)
    bin_df[:] = False
    for ind,val in tqdm.tqdm(df.iterrows()):
        bin_df.loc[ind,val["tissues"]] = True
        
    return bin_df

In [5]:
[tissue_map,tissue_map_short]= pickle.load(open("Results/tissue_map.p","rb"))

IOError: [Errno 2] No such file or directory: 'Results/tissue_map.p'

In [ ]:
f_in = 'Results/tss_annotation/geneID_df_02.p' 
gene = pickle.load(open(f_in,"rb"))
gene["tissues"] = gene["samples"].apply(lambda x: to_collapse(x,tissue_map_short))
gene_bin = create_binary(gene,tissues,colname="tissues")

In [ ]:
## Create CDF of genes
def create_cdf(bin_df):
    ## Steps:
    # 1. Create dictionary of tissue combo to number of genes 
    # 1. Double for loop and get the pairwise union overlap
    # 2. Pop them from list
    # 3. While not done with all columns
         #-Loop through and find the next maximum overlap
         #-Add to dictionary
         #-Pop from list
    cdf_tissues = []
    cdf_number = []
    all_cols = list(bin_df.columns.values)
    max_overlap = 0
    max_combo = ()
    max_num = 0
    for i in all_cols:
        for j in all_cols[1:]:
            overlap =  set((bin_df[bin_df[i]].index)).union(set((bin_df[bin_df[j]].index)))
            if len(overlap) > max_num:
                max_overlap = overlap
                max_num = len(overlap)
                max_combo  = (i,j)



    cdf_tissues.append(max_combo[np.argmax([bin_df[max_combo[0]].sum(),bin_df[max_combo[1]].sum()])])
    cdf_tissues.append(max_combo[np.argmin([bin_df[max_combo[0]].sum(),bin_df[max_combo[1]].sum()])])

    cdf_number.append(max(bin_df[max_combo[0]].sum(),bin_df[max_combo[1]].sum()))
    cdf_number.append(max_num)

    all_cols.remove(max_combo[0])
    all_cols.remove(max_combo[1])

    while len(all_cols) > 0:
        no_addition = True
        for i in all_cols:
            overlap =  max_overlap.union(set((bin_df[bin_df[i]].index)))
            if len(overlap) > max_num:
                max_num = len(overlap)
                max_ind = i
                no_addition = False
        if no_addition:
            break

        max_overlap = max_overlap.union(set((bin_df[bin_df[max_ind]].index)))
        cdf_tissues.append(max_ind)
        cdf_number.append(max_num)
        all_cols.remove(max_ind)
    
    return cdf_number,cdf_tissues
  





In [6]:
cdf_number_gene,cdf_tissues_gene = create_cdf(gene_bin)

NameError: name 'create_cdf' is not defined

In [ ]:
if not os.path.exists("Results/Figures"):
    os.mkdir("Results/Figures")
if not os.path.exists("Results/Figures/cdf_tissues"):
    os.mkdir("Results/Figures/cdf_tissues")


In [ ]:
f, ax = plt.subplots()
plt.scatter(np.arange(1,len(cdf_number_gene)+1),cdf_number_gene)
plt.xlabel("Tissue added")
plt.ylabel("Cumulative Number of Genes Detected")
ax.set_xticks(range(1,len(cdf_tissues_gene)+1));
ax.set_xticklabels(cdf_tissues_gene);
ax.tick_params(axis="x",rotation=45)
#plt.xticks(rotation="vertical");
plt.savefig("Results/Figures/cdf_tissues/cdf_geneID_df_02_tissues.png",bbox_inches="tight")
plt.savefig("Results/Figures/cdf_tissues/cdf_geneID_df_02_tissues.svg",bbox_inches="tight")

## Enhancers:
1. Enhancer:  
   i. Annotation as intergenic  
   ii. Distance to TSS > 1000  
   iii. Need to handle same strand/divergent transcription  
2. Filter:  
   i. > 1 CPM  
   ii. > 10 CPM ??  
   iii. > 2 tissues  

In [ ]:
import annotation
import helper

In [ ]:
## Create binary of expression matrix
def create_binary_from_expr(expr_df,threshold=1):
    bin_df = (expr_df > threshold)
    return bin_df

In [ ]:
intergenic = pd.read_csv("Results/merged/samples.merge.anno",sep="\t",index_col=0)
#intergenic["Annotation"].filna(inplace=True)
intergenic = (intergenic["Annotation"].str.contains("Intergenic") & (np.abs(intergenic["Distance to TSS"].fillna(0).astype(int)) > 1000))

expr_df = pd.read_csv("Results/merged/tissues.merge.peaksexpression.log10",sep="\t",index_col = 0)
expr_df = expr_df[expr_df.index.isin(intergenic.index)]
expr_df.head()

## 1. Threshold = 10 (log10=1)
   ## Distance to gene > 1000

In [ ]:
bin_df_enh = create_binary_from_expr(expr_df,threshold=1)

cdf_number,cdf_tissues = create_cdf(bin_df_enh)

In [ ]:
f, ax = plt.subplots()
plt.scatter(np.arange(1,len(cdf_number)+1),cdf_number)
plt.xlabel("Tissue added")
plt.ylabel("Cumulative Number of enhancer peaks Detected")
ax.set_xticks(range(1,len(cdf_tissues)+1));
ax.set_xticklabels(cdf_tissues);
ax.tick_params(axis="x",rotation=45)
#plt.xticks(rotation="vertical");
plt.savefig("Results/Figures/cdf_tissues/cdf_enhancers_tissues.png",bbox_inches="tight")
plt.savefig("Results/Figures/cdf_tissues/cdf_enhancers_tissues.svg",bbox_inches="tight")

In [ ]:
cdf_tissues